### Entanglement Swapping in a Quantum Network of 3 nodes
#### Implementation of Ekert Protocol in a 3-node Quantum Network using Quantum Repeater for the Entanglement Swapping. The resulting qubits for Alice and Bob are evaluated through a fidelity function, verifying that both qubits are entangled. After that we can create the shared key and encrypt messages with classical cryptography.
_ref: https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater.html (adapted by D-Cryp7 for Netsquid 1.1.6)_  

_TODO: Add more comments for explaining the functionality_

In [1]:
# imports
import pydynaa as pd
import netsquid as ns
from netsquid.components import instructions as instr
from netsquid.components import ClassicalChannel, QuantumChannel
from netsquid.util.simtools import sim_time
from netsquid.util.datacollector import DataCollector
from netsquid.protocols.nodeprotocols import LocalProtocol, NodeProtocol
from netsquid.protocols.protocol import Signals
from netsquid.components.component import Message, Port
from netsquid.nodes.node import Node
from netsquid.nodes.network import Network
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.components.qprocessor import QuantumProcessor
from netsquid.components.qprogram import QuantumProgram
from netsquid.qubits import qubitapi as qapi
from netsquid.qubits import ketstates as ks
from netsquid.qubits.state_sampler import StateSampler
from netsquid.components.models.delaymodels import FixedDelayModel, FibreDelayModel
from netsquid.components.models import DepolarNoiseModel
from netsquid.nodes.connections import DirectConnection
from netsquid.examples.entanglenodes import EntangleNodes
from netsquid.examples.purify import Filter, Distil
from pydynaa import EventExpression

In [2]:
class Repeater(NodeProtocol):
    """Entangles two nodes given both are entangled with an intermediary midpoint node.

    Parameters
    ----------
    node : :py:class:`~netsquid.nodes.node.Node`
        Node to run the repeater or corrector protocol on.
    port : :py:class:`~netsquid.components.component.Port`
        Port to use for classical IO communication between the repeater and
        corrector node.
    role : "repeater" or "corrector"
        Whether this protocol should act as a repeater or a corrector. Both are needed.
    start_expression : :class:`~pydynaa.EventExpression`
        EventExpression node should wait for before starting.
        The EventExpression should have a protocol as source, this protocol should
        signal the quantum memory position of the qubit. In the case of a midpoint
        repeater it requires two such protocols, both signalling a quantum memory
        position
    name : str or None, optional
        Name of protocol. If None a default name is set.

    """
    MSG_HEADER = "repeater:corrections"

    def __init__(self, node, port, role, start_expression=None, name=None):
        if role.lower() not in ["repeater", "corrector"]:
            raise ValueError
        self.role = role.lower()
        name = name if name else "Repeater({}, {})".format(node.name, role)
        super().__init__(node=node, name=name)
        self.start_expression = start_expression
        self.port = port
        # Used by corrector:
        self._correction = None
        self._mem_pos = None

    @property
    def start_expression(self):
        return self._start_expression

    @start_expression.setter
    def start_expression(self, value):
        if value:
            if not isinstance(value, EventExpression):
                raise TypeError("Start expression of the corrector role should be an "
                                "event expression")
            elif self.role == "repeater" and value.type != EventExpression.AND:
                raise TypeError("Start expression of the repeater role should be an "
                                "expression that returns two values.")
        self._start_expression = value

    def run(self):
        if self.role == "repeater":
            yield from self._run_repeater()
        else:
            yield from self._run_corrector()

    def _run_repeater(self):
        # Run loop for midpoint repeater node
        while True:
            evexpr = yield self.start_expression
            assert evexpr.first_term.value and evexpr.second_term.value
            source_A = evexpr.first_term.atomic_source
            source_B = evexpr.second_term.atomic_source
            signal_A = source_A.get_signal_by_event(
                evexpr.first_term.triggered_events[0], self)
            signal_B = source_B.get_signal_by_event(
                evexpr.second_term.triggered_events[0], self)
            # Run bell state measurement program
            measure_program = BellMeasurementProgram()
            pos_A = signal_A.result
            pos_B = signal_B.result
            yield self.node.qmemory.execute_program(measure_program, [pos_A, pos_B])
            m, = measure_program.output["BellStateIndex"]
            # Send measurement to B
            self.port.tx_output(Message([m], header=self.MSG_HEADER))

            self.send_signal(Signals.SUCCESS)

    def _run_corrector(self):
        # Run loop for endpoint corrector node
        port_expression = self.await_port_input(self.port)
        while True:
            evexpr = yield self.start_expression | port_expression
            if evexpr.second_term.value:
                cmessage = self.port.rx_input(header=self.MSG_HEADER)
                if cmessage:
                    self._correction = cmessage.items
            else:
                source_protocol = evexpr.first_term.atomic_source
                ready_signal = source_protocol.get_signal_by_event(
                    event=evexpr.first_term.triggered_events[0], receiver=self)
                self._mem_pos = ready_signal.result
            if self._mem_pos is not None and self._correction is not None:
                yield from self._do_corrections()

    def _do_corrections(self):
        m = self._correction[0]
        if self.node.qmemory.busy:
            yield self.await_program(self.node.qmemory)
        if m == ks.BellIndex.B01 or m == ks.BellIndex.B11:
            self.node.qmemory.execute_instruction(instr.INSTR_X, [self._mem_pos])
        if m == ks.BellIndex.B10 or m == ks.BellIndex.B11:
            self.node.qmemory.execute_instruction(instr.INSTR_Z, [self._mem_pos])
        if self.node.qmemory.busy:
            yield self.await_program(self.node.qmemory)
        self.send_signal(Signals.SUCCESS, self._mem_pos)
        # Reset values
        self._mem_pos = None
        self._correction = None

    @property
    def is_connected(self):
        if self.start_expression is None:
            return False
        if not self.check_assigned([self.node], Node):
            return False
        if not self.check_assigned([self.port], Port):
            return False
        if self.role == "repeater" and (self.node.qmemory is None or
                                        self.node.qmemory.num_positions < 2):
            return False
        if self.role == "corrector" and (self.node.qmemory is None or
                                         self.node.qmemory.num_positions < 1):
            return False
        return True

In [3]:
class BellMeasurementProgram(QuantumProgram):
    """Program to perform a Bell measurement on two qubits.

    Measurement results are stored in output key "BellStateIndex""

    """
    default_num_qubits = 2

    def program(self):
        q1, q2 = self.get_qubit_indices(2)
        self.apply(instr.INSTR_MEASURE_BELL, [q1, q2], inplace=False,
                   output_key="BellStateIndex")
        yield self.run()

In [4]:
class RepeaterExample(LocalProtocol):
    """Protocol for a complete repeater experiment including purification.

    Will run for specified number of times then stop, recording results after each run.

    Parameters
    ----------
    node_A : :py:class:`~netsquid.nodes.node.Node`
        Node to be entangled via repeater.
        Must be specified before protocol can start.
    node_B : :py:class:`~netsquid.nodes.node.Node`
        Node to be entangled via repeater.
        Must be specified before protocol can start.
    node_R : :py:class:`~netsquid.nodes.node.Node`
        Repeater node that will entangle nodes A and B.
        Must be specified before protocol can start.
    num_runs : int
        Number of successful runs to do.
    purify : "filter" or "distil" or None, optional
        Purification protocol to run. If None, no purification is done.
    epsilon : float
        Parameter used in filter's measurement operator.

    Subprotocols
    ------------
    entangle_A : :class:`~netsquid.examples.entanglenodes.EntangleNodes`
        Entanglement generation protocol running on node A to entangle with R.
    entangle_Ra : :class:`~netsquid.examples.entanglenodes.EntangleNodes`
        Entanglement generation protocol running on node R to entangle with A.
    entangle_B : :class:`~netsquid.examples.entanglenodes.EntangleNodes`
        Entanglement generation protocol running on node B to entangle with R.
    entangle_Rb : :class:`~netsquid.examples.entanglenodes.EntangleNodes`
        Entanglement generation protocol running on node R to entangle with B.
    purify_A : :class:`Filter` or :class:`Distil`
        Purification protocol running on node A to purify entanglement with R.
    purify_Ra : :class:`Filter` or :class:`Distil`
        Purification protocol running on node R to purify entanglement with A.
    purify_B : :class:`Filter` or :class:`Distil`
        Purification protocol running on node B to purify entanglement with R.
    purify_Rb : :class:`Filter` or :class:`Distil`
        Purification protocol running on node R to purify entanglement with B.
    repeater_R : :class:`~netsquid.examples.repeater.Repeater`
        Repeater protocol running on node R to do midpoint entanglement swap.
    repeater_B : :class:`~netsquid.examples.repeater.Repeater`
        Repeater protocol running on node B to do endpoint correction.

    """

    def __init__(self, node_A, node_B, node_R, num_runs, purify="filter", epsilon=0.3):
        super().__init__(nodes={"A": node_A, "B": node_B, "R": node_R},
                         name="Repeater with purification example")
        self.num_runs = num_runs
        purify = purify.lower()
        if purify not in ("filter", "distil"):
            raise ValueError("{} unknown purify option".format(purify))
        self._add_subprotocols(node_A, node_B, node_R, purify, epsilon)
        # Set entangle start expressions
        start_expr_ent_A = (self.subprotocols["entangle_A"].await_signal(
            self.subprotocols["purify_A"], Signals.FAIL) |
            self.subprotocols["entangle_A"].await_signal(self, Signals.WAITING))
        self.subprotocols["entangle_A"].start_expression = start_expr_ent_A
        start_expr_ent_B = (self.subprotocols["entangle_B"].await_signal(
            self.subprotocols["purify_B"], Signals.FAIL) |
            self.subprotocols["entangle_B"].await_signal(self, Signals.WAITING))
        self.subprotocols["entangle_B"].start_expression = start_expr_ent_B
        # Set purify start expressions
        self._start_on_success("purify_A", "entangle_A")
        self._start_on_success("purify_Ra", "entangle_Ra")
        self._start_on_success("purify_B", "entangle_B")
        self._start_on_success("purify_Rb", "entangle_Rb")
        # Set repeater start expressions
        self._start_on_success("repeater_B", "purify_B")
        start_expr_repeater = (self.subprotocols["repeater_R"].await_signal(
            self.subprotocols["purify_Ra"], Signals.SUCCESS) &
            self.subprotocols["repeater_R"].await_signal(
                self.subprotocols["purify_Rb"], Signals.SUCCESS))
        self.subprotocols["repeater_R"].start_expression = start_expr_repeater

    def _start_on_success(self, start_subprotocol, success_subprotocol):
        # Convenience method to set subprotocol's start expression to be success of another
        self.subprotocols[start_subprotocol].start_expression = (
            self.subprotocols[start_subprotocol].await_signal(
                self.subprotocols[success_subprotocol], Signals.SUCCESS))

    def _add_subprotocols(self, node_A, node_B, node_R, purify, epsilon):
        # Setup all of the subprotocols
        purify = purify.lower()
        # Add entangle subprotocols
        pairs = 2 if purify == "distil" else 1
        self.add_subprotocol(EntangleNodes(
            node=node_A, role="source", input_mem_pos=0, num_pairs=pairs, name="entangle_A"))
        self.add_subprotocol(EntangleNodes(
            node=node_B, role="source", input_mem_pos=0, num_pairs=pairs, name="entangle_B"))
        self.add_subprotocol(EntangleNodes(
            node=node_R, role="receiver", input_mem_pos=0, num_pairs=pairs, name="entangle_Ra"))
        self.add_subprotocol(EntangleNodes(
            node=node_R, role="receiver", input_mem_pos=1, num_pairs=pairs, name="entangle_Rb"))
        # Add purify subprotocols
        if purify == "filter":
            purify_cls, kwargs = Filter, {"epsilon": epsilon}
        else:
            distil_role = "A"
            purify_cls, kwargs = Distil, {"role": distil_role}
        for node1, node2, name, distil_role in [
                (node_A, node_R, "purify_A", "A"),
                (node_R, node_A, "purify_Ra", "B"),
                (node_B, node_R, "purify_B", "A"),
                (node_R, node_B, "purify_Rb", "B")]:
            self.add_subprotocol(purify_cls(
                node1, port=node1.get_conn_port(node2.ID), name=name, **kwargs))
        # Add repeater subprotocols
        self.add_subprotocol(Repeater(
            node_R, node_R.get_conn_port(node_B.ID), role="repeater", name="repeater_R"))
        self.add_subprotocol(Repeater(
            node_B, node_B.get_conn_port(node_R.ID), role="corrector", name="repeater_B"))

    def run(self):
        self.start_subprotocols()
        for i in range(self.num_runs):
            start_time = sim_time()
            self.subprotocols["entangle_A"].entangled_pairs = 0
            self.subprotocols["entangle_B"].entangled_pairs = 0
            self.send_signal(Signals.WAITING)
            yield (self.await_signal(self.subprotocols["purify_A"], Signals.SUCCESS) &
                   self.await_signal(self.subprotocols["repeater_B"], Signals.SUCCESS))
            signal_A = self.subprotocols["purify_A"].get_signal_result(
                label=Signals.SUCCESS, receiver=self)
            signal_B = self.subprotocols["repeater_B"].get_signal_result(
                label=Signals.SUCCESS, receiver=self)
            result = {
                "pos_A": signal_A,
                "pos_B": signal_B,
                "time": sim_time() - start_time,
                "pairs_A": self.subprotocols["entangle_A"].entangled_pairs,
                "pairs_B": self.subprotocols["entangle_B"].entangled_pairs,
            }
            self.send_signal(Signals.SUCCESS, result)

In [5]:
def example_network_setup(source_delay=1e5, source_fidelity_sq=0.8, depolar_rate=1000,
                          node_distance=20):
    """Create an example network for use with the repeater protocols.

    Returns
    -------
    :class:`~netsquid.components.component.Component`
        A network component with nodes and channels as subcomponents.

    Notes
    -----
        This network is also used by the matching integration test.

    """
    network = Network("Repeater_network")
    state_sampler = StateSampler(
        [ks.b01, ks.s00],
        probabilities=[source_fidelity_sq, 1 - source_fidelity_sq])
    node_a, node_b, node_r = network.add_nodes(["node_A", "node_B", "node_R"])
    # Setup end-node A:
    node_a.add_subcomponent(QuantumProcessor(
        "quantum_processor_a", num_positions=2, fallback_to_nonphysical=True,
        memory_noise_models=DepolarNoiseModel(depolar_rate)))
    source_a = QSource(
        "QSource_A", state_sampler=state_sampler, num_ports=2, status=SourceStatus.EXTERNAL,
        models={"emission_delay_model": FixedDelayModel(delay=source_delay)})
    node_a.add_subcomponent(source_a)
    # Setup end-node B:
    node_b.add_subcomponent(QuantumProcessor(
        "quantum_processor_b", num_positions=2, fallback_to_nonphysical=True,
        memory_noise_models=DepolarNoiseModel(depolar_rate)))
    source_b = QSource(
        "QSource_B", state_sampler=state_sampler, num_ports=2, status=SourceStatus.EXTERNAL,
        models={"emission_delay_model": FixedDelayModel(delay=source_delay)})
    node_b.add_subcomponent(source_b)
    # Setup midpoint repeater node R
    node_r.add_subcomponent(QuantumProcessor(
        "quantum_processor_r", num_positions=4, fallback_to_nonphysical=True,
        memory_noise_models=DepolarNoiseModel(depolar_rate)))
    # Setup classical connections
    conn_cfibre_ar = DirectConnection(
        "CChannelConn_AR",
        ClassicalChannel("CChannel_A->R", length=node_distance,
                         models={"delay_model": FibreDelayModel(c=200e3)}),
        ClassicalChannel("CChannel_R->A", length=node_distance,
                         models={"delay_model": FibreDelayModel(c=200e3)}))
    network.add_connection(node_a, node_r, connection=conn_cfibre_ar)
    conn_cfibre_br = DirectConnection(
        "CChannelConn_BR",
        ClassicalChannel("CChannel_B->R", length=node_distance,
                         models={"delay_model": FibreDelayModel(c=200e3)}),
        ClassicalChannel("CChannel_R->B", length=node_distance,
                         models={"delay_model": FibreDelayModel(c=200e3)}))
    network.add_connection(node_b, node_r, connection=conn_cfibre_br)
    # Setup quantum channels
    qchannel_ar = QuantumChannel(
        "QChannel_A->R", length=node_distance,
        models={"quantum_loss_model": None, "delay_model": FibreDelayModel(c=200e3)})
    port_name_a, port_name_ra = network.add_connection(
        node_a, node_r, channel_to=qchannel_ar, label="quantum")
    qchannel_br = QuantumChannel(
        "QChannel_B->R", length=node_distance,
        models={"quantum_loss_model": None, "delay_model": FibreDelayModel(c=200e3)})
    port_name_b, port_name_rb = network.add_connection(
        node_b, node_r, channel_to=qchannel_br, label="quantum")
    # Setup Alice ports:
    node_a.subcomponents["QSource_A"].ports["qout1"].forward_output(
        node_a.ports[port_name_a])
    node_a.subcomponents["QSource_A"].ports["qout0"].connect(
        node_a.qmemory.ports["qin0"])
    # Setup Bob ports:
    node_b.subcomponents["QSource_B"].ports["qout1"].forward_output(
        node_b.ports[port_name_b])
    node_b.subcomponents["QSource_B"].ports["qout0"].connect(
        node_b.qmemory.ports["qin0"])
    # Setup repeater ports:
    node_r.ports[port_name_ra].forward_input(node_r.qmemory.ports["qin0"])
    node_r.ports[port_name_rb].forward_input(node_r.qmemory.ports["qin1"])
    return network

In [6]:
def example_sim_setup(node_A, node_B, node_R, num_runs, purify="filter", epsilon=0.3):
    """Example simulation setup of repeater protocol.

    Returns
    -------
    :class:`~netsquid.examples.repeater.RepeaterExample`
        Example protocol to run.
    :class:`pandas.DataFrame`
        Dataframe of collected data.

    """
    repeater_example = RepeaterExample(
        node_A, node_B, node_R, num_runs=num_runs, purify=purify, epsilon=0.3)
    
    def random_basis():
        from random import randint
        r = randint(0, 1)
        return ns.Z if r else ns.X
    def measure(q, obs):
        measurement_result, prob = ns.qubits.measure(q, obs)
        if measurement_result == 0:
            state = "|0>" if obs == ns.Z else "|+>"
        else:
            state = "|1>" if obs == ns.Z else "|->"
        # print(f"Measured {state} with probability {prob:.1f}")
        return ("Z" if obs == ns.Z else "X"), measurement_result, prob
    
    def record_run(evexpr):
        # Record a repeater run
        protocol = evexpr.triggered_events[-1].source
        result = protocol.get_signal_result(Signals.SUCCESS)
        # Record fidelity
        q_a, = node_A.qmemory.pop(positions=[result["pos_A"]])
        a_basis = random_basis()
        q_b, = node_B.qmemory.pop(positions=[result["pos_B"]])
        b_basis = random_basis()
        f2 = qapi.fidelity([q_a, q_b], ks.b00, squared=True)
        a_measure = measure(q_a, a_basis)
        b_measure = measure(q_b, b_basis)
        return {"F2": f2,
                "pairs_A": result["pairs_A"],
                "A_basis": a_measure[0],
                "A_measure": a_measure[1],
                "pairs_B": result["pairs_B"],
                "B_basis": b_measure[0],
                "B_measure": b_measure[1],
                "time": result["time"]}

    dc = DataCollector(record_run, include_time_stamp=False, include_entity_name=False)
    dc.collect_on(pd.EventExpression(
        source=repeater_example, event_type=Signals.SUCCESS.value))
    return repeater_example, dc

In [51]:
ns.sim_reset()
network = example_network_setup()
repeater_example, dc = example_sim_setup(network.get_node("node_A"),
                                         network.get_node("node_B"),
                                         network.get_node("node_R"),
                                         num_runs=256, purify = "filter") # key length of 256 bits (ideally)
repeater_example.start()
ns.sim_run()
print("Average fidelity of generated entanglement via a repeater and with "
      "filtering: {}".format(dc.dataframe["F2"].mean()))

Average fidelity of generated entanglement via a repeater and with filtering: 0.462890625


/home/kali/anaconda3/envs/sage/lib/python3.8/site-packages/netsquid/util/datacollector.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._dataframe = self._dataframe.append(self._data_buffer, ignore_index=True, sort=False)


In [52]:
df = dc.dataframe # get dataframe that summarizes the number of runs 
df.head()

,F2,pairs_A,A_basis,A_measure,pairs_B,B_basis,B_measure,time
0,1.0,2,X,1,2,Z,0,400000.0
1,0.0,1,X,1,11,X,1,2500000.0
2,0.0,3,Z,0,6,X,1,1200000.0
3,0.0,4,Z,0,1,Z,1,600000.0
4,0.5,6,X,0,2,X,0,1800000.0


In [53]:
df_match = df[df["A_basis"] == df["B_basis"]] # equal basis in both sides
df_fidelity = df_match[df_match["F2"] == 1] # filter with a fidelity of 100% (qubits entangled)
df_fidelity.head()

,F2,pairs_A,A_basis,A_measure,pairs_B,B_basis,B_measure,time
10,1.0,1,Z,1,3,Z,1,900000.0
13,1.0,1,Z,1,1,Z,1,300000.0
16,1.0,2,X,0,1,X,0,400000.0
26,1.0,1,Z,0,1,Z,0,300000.0
27,1.0,5,X,1,4,X,1,1000000.0


In [54]:
df_fidelity.shape[0] # resulting key length

58

In [61]:
# Crypto part (AES - Symmetric Encryption)
from hashlib import sha256
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

def encrypt(key_bits: list, m: bytes):
    strk = ''.join([str(i) for i in key_bits])
    key = bytes(int(strk[i : i + 8], 2) for i in range(0, len(strk), 8))
    key_hash = sha256(key).digest()
    cipher = AES.new(key_hash, AES.MODE_ECB) # Mode ECB just for testing
    c = cipher.encrypt(pad(m, 16))
    return c.hex()

In [66]:
A_key, B_key = list(df_fidelity["A_measure"]), list(df_fidelity["B_measure"])
c = encrypt(A_key, b"S3CR3T_M3SS4G3!!")
c

'0d9979f01a9f675a51ca924d15e7b093c8955dbaf3cde1559c641e0bea4c09b3'

In [68]:
# Decrypting Alice encrypted message

def decrypt(key_bits: list, c: hex):
    strk = ''.join([str(i) for i in key_bits])
    key = bytes(int(strk[i : i + 8], 2) for i in range(0, len(strk), 8))
    key_hash = sha256(key).digest()
    cipher = AES.new(key_hash, AES.MODE_ECB) # Mode ECB just for testing
    m = cipher.decrypt(bytes.fromhex(c))
    return unpad(m, 16)

In [71]:
m = decrypt(B_key, c)
m

b'S3CR3T_M3SS4G3!!'

In [ ]:
# qapi.fidelity? # for research :p